In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go



Getting NASDAQ weekly data

In [2]:
symbol = "^CNXIT"

nq_weekly_df = yf.download(symbol, period="max", interval="1wk")
nq_weekly_df.reset_index(inplace=True)
nq_weekly_df.index = pd.to_datetime(nq_weekly_df["Date"], errors="coerce")

nq_weekly_df

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2007-09-17,2007-09-17,4645.850098,4700.750000,4476.850098,4603.850098,4603.850098,0
2007-09-24,2007-09-24,4605.700195,4882.850098,4520.950195,4804.200195,4804.200195,0
2007-10-01,2007-10-01,4813.350098,4943.649902,4696.000000,4868.450195,4868.450195,0
2007-10-08,2007-10-08,4887.250000,5187.350098,4802.200195,4877.850098,4877.850098,0
2007-10-15,2007-10-15,4891.750000,5037.500000,4572.200195,4894.049805,4894.049805,0
...,...,...,...,...,...,...,...
2023-09-25,2023-09-25,32881.351562,32960.949219,31514.900391,31784.400391,31784.400391,89300
2023-10-02,2023-10-02,31772.750000,32438.400391,31412.500000,32341.699219,32341.699219,65700
2023-10-09,2023-10-09,32216.150391,32981.898438,31434.099609,31807.949219,31807.949219,122600


Getting the Bias of the week. Bias = Bullish or Bearish candle

In [3]:
def get_weekly_bias(x):
    if x["Close"] < x["Open"]:
        return -1
    elif x["Close"] > x["Open"]:
        return 1

In [4]:
nq_weekly_df['weekly_bias'] = nq_weekly_df.apply(get_weekly_bias, axis=1)

nq_weekly_df.fillna(1, inplace=True)



In [5]:
def calculate_high_low_of_week_day(nq):
    days_offset = 5
    week_start_date_object = datetime.strptime(str(nq["Date"]), "%Y-%m-%d %H:%M:%S")
    week_end_date_object = week_start_date_object + timedelta(days=days_offset)
    nq_daily_df = yf.download(
        symbol,
        start=week_start_date_object,
        end=week_end_date_object,
        interval="1d",
        progress=False,
    )
    nq_daily_df.fillna(1, inplace=True)

    nq_daily_df.reset_index(inplace=True)

    if nq["weekly_bias"] == 1:
        day_of_the_week_list = [
            nq_daily_df[nq_daily_df["Low"] == nq["Low"]]["Date"]
            .dt.strftime("%A")
            .values
        ][0].tolist()
        if len(day_of_the_week_list):
            return day_of_the_week_list[0]
    elif nq["weekly_bias"] == -1:
        day_of_the_week_list = [
            nq_daily_df[nq_daily_df["High"] == nq["High"]]["Date"]
            .dt.strftime("%A")
            .values
        ][0].tolist()
        if len(day_of_the_week_list):
            return day_of_the_week_list[0]

In [6]:
nq_weekly_df["week_high_low_day"] = nq_weekly_df.apply(
    calculate_high_low_of_week_day, axis=1
)
nq_weekly_df.fillna("Monday", inplace=True)



1 Failed download:
['^CNXIT']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2016-04-11 00:00:00 -> 2016-04-16 00:00:00)')


AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
nq_weekly_df.to_csv(f'./csv/weekly_{symbol}.csv')


In [ ]:
bullish_days = nq_weekly_df.loc[nq_weekly_df['weekly_bias'] == 1]
bearish_days = nq_weekly_df.loc[nq_weekly_df['weekly_bias'] == -1]


fig = go.Figure()
fig.add_trace(go.Histogram(histfunc="count", y=bullish_days["weekly_bias"], x=bullish_days["week_high_low_day"], name="weekly_bullish_count"))
fig.add_trace(go.Histogram(histfunc="count", y=bearish_days["weekly_bias"], x=bearish_days["week_high_low_day"], name="weekly_bearish_count"))

fig.show()

In [ ]:


nq_daily_df = yf.download(symbol, period="max", interval="1d")
nq_daily_df.reset_index(inplace=True)
nq_daily_df.index = pd.to_datetime(nq_daily_df["Date"], errors="coerce")



[*********************100%%**********************]  1 of 1 completed


In [ ]:

nq_daily_df['dr'] = nq_daily_df.apply(lambda x: x["High"] - x["Low"], axis=1)

nq_daily_df['adr'] = nq_daily_df['dr'].rolling(20).mean()

nq_daily_df



,Date,Open,High,Low,Close,Adj Close,Volume,dr,adr
Date,,,,,,,,,
2007-09-17,2007-09-17,4518.450195,4549.049805,4482.850098,4494.649902,4494.649902,0,66.199707,NaN
2007-09-18,2007-09-18,4494.100098,4551.799805,4481.549805,4546.200195,4546.200195,0,70.250000,NaN
2007-09-19,2007-09-19,4550.250000,4739.000000,4550.250000,4732.350098,4732.350098,0,188.750000,NaN
2007-09-20,2007-09-20,4734.850098,4760.850098,4721.149902,4747.549805,4747.549805,0,39.700195,NaN
2007-09-21,2007-09-21,4752.950195,4855.700195,4733.700195,4837.549805,4837.549805,0,122.000000,NaN
...,...,...,...,...,...,...,...,...,...
2023-10-20,2023-10-20,19542.150391,19593.800781,19518.699219,19542.650391,19542.650391,198300,75.101562,128.937598
2023-10-23,2023-10-23,19521.599609,19556.849609,19257.849609,19281.750000,19281.750000,176000,299.000000,136.830078
2023-10-25,2023-10-25,19286.449219,19347.300781,19074.150391,19122.150391,19122.150391,225300,273.150391,143.857617


In [ ]:
px.scatter(data_frame=nq_daily_df, x="Date", y='adr', trendline='ols' )

c:\Users\praveenjaisankar\AppData\Local\pypoetry\Cache\virtualenvs\metatrader-Cdlomroj-py3.10\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\praveenjaisankar\AppData\Local\pypoetry\Cache\virtualenvs\metatrader-Cdlomroj-py3.10\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

